# Geospatial Analysis of Internet Access Disparities

In this notebook, we will work with real crowdsourced Internet measurement data to identify regional boundaries for sampling Internet latency. The techniques used in this notebook are very similar to those used by the government to figure out boundaries for census tracts, zip codes and neighborhoods in a city.

We will be implementing the below clustering pipeline. If you're interested in learning more, we encourage you to check out [this paper](https://arxiv.org/pdf/2405.11138).

![Pipeline](pipeline.png)

# Install packages

In [ ]:
!pip3 install pandas numpy scikit-learn seaborn h3 geopandas pysal shapely

# 1. Spatial Interpolation

The measurement data for this assignment consists of [Ookla](https://speedtest.net) speed tests conducted in Chicago over the period of January 2022 - May 2023. For simplicity, we will only consider tests conducted by users that subscribed to Comcast, a popular Internet Service Provider (ISP) in the United States. This data typically suffers from a self-selection bias, meaning that we obtain a new measurement only when an Internet user somewhere in Chicago decides to measure their Internet speed. This causes some geographical areas of Chicago to be sparsely sampled in the dataset, causing problems for network operators and regulators to get a real assessment of Internet performance in these areas. 

To address this sparsity problem, we make an assumption that users conducting tests from nearby regions likely share the same Internet infrastructure, and therefore, experience similar latency. This assumption opens up a world of possibilities. We can now apply spatial interpolation techniques to fill these data gaps by "borrowing" data from nearby locations. A widely used algorithm for doing so is called [Inverse Distance Weighting (IDW)](https://en.wikipedia.org/wiki/Inverse_distance_weighting).

In this part, we will implement IDW from scratch and use it to estimate latency along a granular grid of points that covers all of Chicago.

## 1.1. Data loading and preprocessing

First, load the dataset (`data/Chicago_speedtests.csv`) into a pandas dataframe.

You will notice that the locations (longitude and latitude columns) are in degrees. The IDW algorithm typically uses Euclidean distances, which are calculated in units such as miles or meters. Working with degrees is problematic because a distance of 1 degree can represent different distances (in meters or miles) near the equator than on the poles.

To solve this problem, we will work with a projected coordinate reference system (CRS) as shown below.

In [ ]:
# Convert the dataframe to a GeoDataFrame (Assuming that you loaded the data in a dataframe called df)

import geopandas as gpd

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# First set a standard CRS for the GeoDataFrame. This is done because the GeoDataFrame does not have a CRS yet.
gdf = gdf.set_crs(epsg=4326)

# Convert to EPSG 32616, which is the recommended projection for the Chicago area
gdf = gdf.to_crs(epsg=32616)

# Re-assign the new coordinates as x and y columns

gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

gdf

Now, the columns `x` and `y` in `gdf` contain the projected coordinates. We will use these coordinates to implement IDW.

## 1.2. Target point generation

Now, we will generate target points for interpolation, i.e, points where we want latency estimates to be calculated by IDW. 

The functions provided below help us generate target points at a predefined grid separation. Implement the `generate_grid_points` function by following the hints in the comments.

In [ ]:
from shapely.wkt import loads
from shapely.geometry import Point
import numpy as np

def load_chicago_boundaries():
    bdry = pd.read_csv('data/City_boundaries.csv')
    bdry['the_geom'] = bdry['the_geom'].apply(loads)
    bdry = gpd.GeoDataFrame(bdry, geometry='the_geom')
    bdry = bdry.set_crs(epsg=4326)
    bdry = bdry.to_crs(epsg=32616)
    return bdry

def generate_grid_points(bdry, spacing_in_meters):
    # TODO: Get the coordinates for a bounding box (a rectangle) around the city (Hint: Use the `total_bounds` attribute of the GeoDataFrame)

    # TODO: Generate grid points within the bounding box at the specified spacing

    # Collect the grid points that are actually within the city boundaries (Hint: Use the shapely Point object and geopandas' `contains` method for filtering)
    grid_points_within_city = []

    # TODO: Perform the filtering here

    # Return the grid points
    return grid_points_within_city

def plot_grid_points(bdry, grid_points):
    ax = bdry.plot(color='white', edgecolor='black')
    for x, y in grid_points:
        ax.plot(x, y, 'ro', markersize=0.5)
    ax.axis('off')

Use the above functions to first generate the grid points at a separation of 200 meters. Then, plot the points on the map of Chicago.

## 1.3. IDW Implementation

IDW assigns weights to each nearby data point based on its distance from an unsampled location. It uses these weights to calculate a linear combination of nearby values as an estimate of the target metric at an unsampled location. The relationship between the similarity of nearby data points and their distance is assumed to be inverse in nature.

At an unsampled location $(x, y)$, the IDW estimate of latency is given by:

$z(x, y) = \frac{\sum_{i = 1}^{n} \frac{z_i}{d_i^p}}{\sum_{i = 1}^{n} \frac{1}{d_i^p}}$

where:

$z(x, y)$: The latency estimate at $(x, y)$.

$n$: Number of source points

$d_i$: Euclidean distance between $(x, y)$ and the $i^{th}$ source point. Remember that the Euclidean distance between $(x_1, y_1)$ and $(x_2, y_2)$ is given by $\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$.

$z_i$: Latency at the $i^{th}$ source point

$p$: A power parameter that controls the influence of distance on latency

Implement the function below.

In [ ]:
def idw(src_x, src_y, target_x, target_y, src_z, power=2):
    """
    Inverse Distance Weighting (IDW) interpolation algorithm.

    Parameters:
    src_x (array): The x-coordinates of the source points.
    src_y (array): The y-coordinates of the source points.
    target_x (array): The x-coordinates of the target points.
    target_y (array): The y-coordinates of the target points.
    values (array): The values associated with the source points.
    power (float): The power parameter for the inverse distance weighting.
    
    Returns:
        target_z: An array of interpolated values at the target points.

    Hint: Iterate over target points and then use numpy for vectorized operations.
    """
    pass

Now, use the `idw` function to estimate latency at the grid points generated in 1.2. Use a power parameter of 2.

# 2. Hexagon Overlay

We now have latency estimates at a granular level all over Chicago. Next, we will bin these points into tiny hexagonal cells. Instead of clustering individual points, we will later cluster entire cells that these points fall into. This makes sure that our boundaries are less influenced by latency at individual locations, and more by groups of locations falling in the same area.

To bin individual locations into cells, we will use the H3 library. This library was originally created by developers at Uber for taxi demand forecasting.

Now, assign a `hex_id` to each grid location using the H3 library.

**Approach:**

1. Put the grid locations and latency estimates in a single GeoDataFrame. **[Implemented]**
2. Convert the grid locations to spherical coordinates (longitude and latitude, in degrees). **[Implemented]**
3. Use an appropriate H3 function to assign a hexagon to each grid location. Check H3 documentation [here](https://h3geo.org/docs). You will notice that the function call requires a resolution value to be specified. Use `resolution=8`. **[TODO]**
4. Add the `hex_id` column to the original dataframe and obtain the coordinates for hexagon boundaries. **[Implemented]** 

In [ ]:
import h3
from shapely.geometry import Polygon

def invert_tuples(tuple_of_tuples):
    return [(p[1], p[0]) for p in tuple_of_tuples]

def hexbin(grid_x, grid_y, grid_z, resolution=8):
    """
    Perform binning of the grid points using the H3 library.

    Parameters:
    grid_x (array): The x-coordinates of the grid points.
    grid_y (array): The y-coordinates of the grid points.
    grid_z (array): The values associated with the grid points.
    resolution (int): The resolution of the hexagonal grid.

    Returns:
        hexagons: A list of hexagon IDs.
    """
    # Build a GeoDataFrame with the grid points and latency values
    points = pd.DataFrame({'x': grid_x, 'y': grid_y, 'z': grid_z})
    points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.x, points.y))
    points = points.set_crs(epsg=32616)
    points = points.to_crs(epsg=4326) # Convert to lat-long
    x_new = points.geometry.x.values
    y_new = points.geometry.y.values
    hexagons = []
    ############################################################
    # TODO: Assign each grid point to a hexagon
    ############################################################
    # Add the hex_id column in the GeoDataFrame
    points['hex_id'] = hexagons
    # Get hexagon boundaries in a column
    points['hex_boundary'] = points['hex_id'].apply(lambda x: h3.cell_to_boundary(x))
    points['hex_boundary'] = points['hex_boundary'].apply(invert_tuples)
    points['hex_boundary'] = points['hex_boundary'].apply(lambda x: Polygon(x))
    points = gpd.GeoDataFrame(points, geometry='hex_boundary')
    return points

Call the above function and obtain a GeoDataFrame with assigned `hex_id`s.

# 3. Spatial Clustering

## 3.1 Aggregation

Now, we will aggregate latency locally within the hexagon cells to get an assessment of regional performance. We will cluster these aggregates later on to obtain the boundaries. 

Pick a suitable function for aggregating latency over each cell. Examples could be `mean`, `median`, `standard deviation`, `percentiles`, etc. Also write a justification for why you chose a particular function.

You're also welcome to pick more than one function and compare your choices later on.

Calculate your aggregate(s) in the below cell. Consider joining the aggregated dataframe back with the `points` dataframe to retain the `hex_boundary` column. This column will later be useful for clustering.

This code block just ensures that we're working with a GeoDataFrame. We will set the geometry column as `hex_boundary`.

In [ ]:
# Assuming that your aggregated dataframe is called `agg_df`

agg_df = gpd.GeoDataFrame(agg_df, geometry='hex_boundary')
agg_df = agg_df.set_crs(epsg=4326)

## 3.2 Clustering Implementation

Below function builds clusters for the choice of two parameters: `n_clusters` and `floor`. `n_clusters` refers to the number of clusters to be formed, and `floor` controls the minimum number of hexagons in a cluster.

We have implemented the `cluster_skater` function for you.

In [ ]:
import spopt
import libpysal
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise as skm
import pickle
import os

def cluster_skater(agg_df, metric, n_clusters, floor):
    """
    Perform clustering of the hexagons using the SKATER algorithm.

    Parameters:
    agg_df (GeoDataFrame): A GeoDataFrame containing the aggregated data.
    metric (str): The column name of the metric to use for clustering.
    n_clusters (int): The number of clusters to create.
    floor (int): The minimum number of hexagons in each cluster.

    Returns:
        clusters: A GeoDataFrame containing the clusters.
    """
    w = libpysal.weights.Queen.from_dataframe(agg_df, use_index=False) # Create spatial weights. Used for encoding spatial relationships between hexagons.
    spanning_forest_kwds = dict(
        dissimilarity=skm.manhattan_distances,
        affinity=None,
        reduction=np.sum,
        center=np.mean,
        verbose=False
    )
    model = spopt.region.Skater(
        agg_df,
        w,
        [metric],
        n_clusters=n_clusters,
        floor=floor,
        trace=False,
        islands="increase",
        spanning_forest_kwds=spanning_forest_kwds
    )
    model.solve()
    clusters = agg_df.copy()
    clusters = clusters.assign(clusters=model.labels_)
    clusters['clusters'] = clusters['clusters'].astype(str)
    return clusters

Choose a `floor` of 50 and plot your clusters while raising `n_clusters` from 2 to 11. Beyond what value of `n_clusters` do you observe little or no change in the clusterings?

Now implement the below function that uses the KMeans algorithm for building clusters. Using the same `n_clusters` as the answer for the previous part, plot and compare the differences between your clusters for KMeans and SKATER. Write your observations in a markdown cell. Think about spatial contiguity and geographic patterns while preparing your answer.

In [2]:
from sklearn.cluster import KMeans

def cluster_kmeans(agg_df, metric, n_clusters):
    """
    Perform clustering of the hexagons using the KMeans algorithm.

    Parameters:
    agg_df (GeoDataFrame): A GeoDataFrame containing the aggregated data.
    metric (str): The column name of the metric to use for clustering.
    n_clusters (int): The number of clusters to create.

    Returns:
        clusters: A GeoDataFrame containing the clusters.
    """
    pass